In [3]:
import numpy as np
import bilby
import lal
import gwpy
import copy
from gwpy.timeseries import TimeSeries
import multiprocessing as mp

import sys
sys.path.append("..")

from create_post_dict import create_post_dict, extract_relevant_info

In [4]:
def call_data_GWOSC(logger, args, calibration, samples, detectors, start_time, end_time, sampling_frequency, roll_off, minimum_frequency, maximum_frequency, psds_array=None, plot=False):
    
    ifo_list = bilby.gw.detector.InterferometerList([])
    
    # define interferometer objects
    for det in detectors:   
        logger.info("Downloading analysis data for ifo {}".format(det))
        ifo = bilby.gw.detector.get_empty_interferometer(det)
        
        channel_type = args['channel_dict'][det]
        channel = f"{det}:{channel_type}"
        
        kwargs = dict(
            start=start_time,
            end=end_time,
            verbose=False,
            allow_tape=True,
        )

        type_kwargs = dict(
            dtype="float64",
            subok=True,
            copy=False,
        )
        data = gwpy.timeseries.TimeSeries.get(channel, **kwargs).astype(
                **type_kwargs)
        
        # Resampling timeseries to sampling_frequency using lal.
        lal_timeseries = data.to_lal()
        lal.ResampleREAL8TimeSeries(
            lal_timeseries, float(1/sampling_frequency)
        )
        data = TimeSeries(
            lal_timeseries.data.data,
            epoch=lal_timeseries.epoch,
            dt=lal_timeseries.deltaT
        )
    
        # define some attributes in ifo
        ifo.strain_data.roll_off = roll_off
        ifo.maximum_frequency = maximum_frequency
        ifo.minimum_frequency = minimum_frequency
        
        # set data as the strain data
        ifo.strain_data.set_from_gwpy_timeseries(data)
        
        # compute the psd
        if det in psds_array.keys():
            print("Using pre-computed psd from results file")
            ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
            frequency_array=psds_array[det][: ,0], psd_array=psds_array[det][:, 1]
            )
        else:
            print('Error: PSD is missing!')
            exit()

        
        if calibration is not None:
            print(f'{det}: Using pre-computed calibration model')
            model = bilby.gw.calibration.Precomputed
            det = ifo.name
            file_paths={"H1":"/home/daniel.williams/events/O3/event_repos/GW150914/C01_offline/calibration/H1.dat",
                        "L1":"/home/daniel.williams/events/O3/event_repos/GW150914/C01_offline/calibration/L1.dat", }
            
            ifo.calibration_model = model.from_envelope_file(
                file_paths[det],
                frequency_array=ifo.frequency_array[ifo.frequency_mask],
                n_nodes=10,
                label=det,
                n_curves=1000,
            )
        

        ifo_list.append(ifo)

    return ifo_list

In [5]:
logger = bilby.core.utils.logger

# adds in detectors and the specs for the detectors. 
waveform = 'C01:IMRPhenomXPHM'
file_path = '/home/shunyin.cheung/GWOSC_posteriors/IGWN-GWTC2p1-v2-GW170818_022509_PEDataRelease_mixed_cosmo.h5'

samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']
end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

Jitter time setting cannot be found. Use default setting.


In [6]:
calibration = None

ifo_list = call_data_GWOSC(logger, args, 
                            calibration, samples, 
                            detectors, start_time, 
                            end_time, sampling_frequency, 
                            roll_off, minimum_frequency, 
                            maximum_frequency, psds_array=psds)

20:52 bilby INFO    : Downloading analysis data for ifo L1
20:52 bilby INFO    : Downloading analysis data for ifo H1


Using pre-computed psd from results file


20:52 bilby INFO    : Downloading analysis data for ifo V1


Using pre-computed psd from results file
Using pre-computed psd from results file


In [8]:
amplitude = 1.0

priors2 = copy.copy(priors)
priors3 = copy.copy(priors)

waveform_name = args['waveform_approximant']
    
# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )
)

proposal_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = args['distance_marginalization_lookup_table'],
    #distance_marginalization_lookup_table = 'TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
    calibration_marginalization=False,
    #calibration_lookup_table=calibration_lookup_table,
)

file_paths={"H1":"/home/daniel.williams/events/O3/event_repos/GW150914/C01_offline/calibration/H1.dat",
                            "L1":"/home/daniel.williams/events/O3/event_repos/GW150914/C01_offline/calibration/L1.dat",}

if calibration is not None:
    for i in range(len(detectors)):

        
        ifo_list[i].calibration_model = bilby.gw.calibration.Precomputed.from_envelope_file(
                file_paths[f"{ifo_list[i].name}"],
                frequency_array=ifo_list[i].frequency_array[ifo_list[i].frequency_mask],
                n_nodes=10,
                label=ifo_list[i].name,
                n_curves=1000,
            )
             

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = args['distance_marginalization_lookup_table'],
    #distance_marginalization_lookup_table = 'TD.npz',
    jitter_time=True,
    priors = priors3,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
    calibration_marginalization=False,
    #calibration_lookup_table=calibration_lookup_table,
)

reference_dict = {'geocent_time': priors2['geocent_time'],
                'luminosity_distance': priors2['luminosity_distance']}

for i in range(1):    
    GWOSC_likelihood_values = samples['log_likelihood'].iloc[i]

    proposal_likelihood.parameters.update(samples.iloc[i].to_dict())
    proposal_likelihood.parameters.update(reference_dict)
    proposal_likelihood_values = proposal_likelihood.log_likelihood_ratio()

    target_likelihood.parameters.update(samples.iloc[i].to_dict())
    target_likelihood.parameters.update(reference_dict)
    target_likelihood_values = target_likelihood.log_likelihood_ratio()

    #ln_weights = target_likelihood_values - proposal_likelihood_values
    
    ln_weights = target_likelihood_values - GWOSC_likelihood_values

    print("difference in log likelihood", ln_weights)

17:15 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
17:15 bilby INFO    : Loaded distance marginalisation lookup table from /home/daniel.williams/events/O3/o3a/run_directories/GW150914/'TD.npz'.npz.
17:15 bilby INFO    : Loaded distance marginalisation lookup table from /home/daniel.williams/events/O3/o3a/run_directories/GW150914/'TD.npz'.npz.


difference in log likelihood -0.9159724244584027
